## 모의고사 1
 학원 규모가 커지면서 시스템을 확장하려 한다. 확장 전 학원의 등록된 학생들의 주요 정보를 파악하려 한다.

### 사전 정보
enrollment_info.csv

|컬럼|정의|type|
|:---|:---|:---:|
|enrollment_code|학생 등록 코드(길이6 알파벳 대문자)|object|
|gender|여자면 F, 남자면 M|object|
|bonus|상점|float64|
|penalty|벌점|float64|
|school_code|소속 학교 코드|object|  

정답 및 해설 : https://tjd229.tistory.com/20

In [32]:
import numpy as np
import pandas as pd


In [33]:
df = pd.read_csv("../content/tjd229/enrollment_info.csv")
print(df.dtypes)
print(df.shape)
df.head()

enrollment_code     object
gender              object
bonus              float64
penalty            float64
school_code         object
dtype: object
(86550, 5)


,enrollment_code,gender,bonus,penalty,school_code
0,QEHRNZ,F,0.0,0.0,SWHS
1,IUGTFR,F,1.6,0.0,PYHS
2,PWJCQA,M,0.0,0.0,NKHS
3,ZYFHCX,F,5.5,0.0,NKHS
4,KIBAHX,M,0.6,0.0,MWHS


### Q1. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.
단계 1 : 각 학생들의 상점(bonus)에서 벌점(penalty)을 뺀 값을 합계 점수로 정의한다.  
단계 2 : 각 학교 코드(school_code)별 합계 점수 평균을 구하고, 이 중 3번째로 큰 값을 A라고 한다.  
단계 3 : 0미만의 합계 점수는 0으로 변경 후, 각 학교 코드(school_code)별 합계 점수 평균을 구하고, 이 중 3번째로 큰 값을 B라고 한다.  

단계 2에서 구한 A와 단계 3에서 구한 B의 절대값 차이를 구하시오  
※ 결과는 반올림하여 소수점 첫째 자리까지 기술하시오. (답안예시 : 22.9)


In [34]:
df1 = df.copy()
df1['total'] = df1['bonus'] - df1.penalty

A = df1.groupby('school_code').total.mean().sort_values(ascending=False)[2]     # .iat[2,] 와 같다. 
A



1.8854451935775354

In [35]:
df1.groupby('school_code').total.mean().index

# groupby 된 column 명으로 index가 생성된다. 

Index(['HJHS', 'MWHS', 'NKHS', 'PYHS', 'SSHS', 'SWHS'], dtype='object', name='school_code')

In [36]:

print(df1.loc[df1['total']<0,:].total.count())

df1.loc[df1['total']<0, 'total'] = 0
# df1['total'] = np.where(df1['total']<0, 0, df1['total'] )

print(df1[df1['total']<0].shape)

B = df1.groupby('school_code').total.mean().sort_values(ascending=False)[2]  # .iat[2,] 와 같다.


7678
(0, 6)


In [37]:
abs(A-B).round(1)

0.1

### Q2. 데이터에는 총 6종류의 학교 코드(school_code)가 있다. 학생들의 등록 코드(enrollment_code)에 학교 코드(school_code)가 포함되어 있는 경우, 시스템에 문제가 발생할 수 있어, 이를 사전에 방지하고자 한다. 등록 코드(enrollment_code)에 학교 코드(school_code)가 포함되어 있는 학생들의 수 총합을 구하시오


※ 학교 코드(school_code)가 **ZXCV**일 때, 등록 코드(enrollment_code)가 **ZXCV**AB, A**ZXCV**B, CD**ZXCV** 등의 경우는 등록 코드(enrollment_code)에 학교 코드(school_code)가 포함되어 있다고 본다.  
※ 학교 코드(school_code)가 **ZXCV**일 때, 등록 코드(enrollment_code)가 **ZX**AB**CV**의 경우는 등록 코드(enrollment_code)에 학교 코드(school_code)가 포함되어 있다고 보지 않는다.  



👉  df.series.str.contains()

In [38]:
df2 = df.copy()

schools = df2['school_code'].unique().tolist()
ans = 0


for name in schools:
    mask = df2['enrollment_code'].str.contains(name)     # 유닉크한 school_code (6개) * enrollment_code의 모든 row (Serise) -> boolean Series 86550개, 6번
    ans+=mask.sum()                                      # mask (boolean series) 의 sum() 은 True 갯수
print(ans)

print(type(df2['enrollment_code'].str.contains(name)))   # boolean series 을 리턴한다. 

# school_code의 리스트를 만들어 각 행(row) enrollment_code에 리스트 값이 있는지 확인 
# 즉, shool_code 중 하나라도 어떤 enrollment_code 에도 포함되지 않아야 한다.  
# df2.loc[df2.loc[:,'enrollment_code'].str.contains(schools)] -> contains() 는 리스트를 처리할 수 없다.
# True 갯수 : https://stackoverflow.com/questions/53550988/count-occurrences-of-false-or-true-in-a-column-in-pandas


3
<class 'pandas.core.series.Series'>


- 👉 다른 방식의 문제 대응
해당 행의 school_code 의 값만 동일 해당 행의 enrollmetn_code 에 있는지 확인  
참고 사이트  
http://ltlkodae.tistory.com/10   
http://bio-info.tistory.com/149  

- iterrows()  

row[0] : index  
row[1] : column  

for indx, colx in df.iterrow()

In [39]:

df2_1 = df.copy()

ans2 = 0
ans2_list=[]

for indX, colX in df2_1.iterrows():
    if colX.school_code in colX.enrollment_code:     # row[1].school_code , row[1].enrollment_code
        ans2 = ans2 +1 
        ans2_list.append(indX)                       # row[0]
print(ans2)
print(ans2_list)

# itertuples() 에 비해 엄첨느리다. 
# 더이상 볼일이 없을까? 

0
[]


- itertuples()  

row.Index : 인덱스   
row.col   : column  

for row in df.itertupes()


In [40]:

df2_2 = df.copy()

ans3 = 0
ans3_list =[]
df2_2['result'] = 0

for row in df2_2.itertuples():
    if row.school_code in row.enrollment_code:  # row.school_code 는 <class 'str'> 타입 이라서 "in" 사용 (contains() 못씀)
        ans3 = ans3 +1
        # ans3_list.append(row.Index)             # 해당 행을 list 에 업데이트 한다. ( index 가 아니다.)
        df2_2.loc[row.Index,'result'] = True      # 해당 행 'result' col 에 결과 입력
print(ans3)            
print(df2_2['result'].sum())

print(ans3_list)                                # list 출력
df2_2.loc[ans3_list]                            # lsit 을 이용해 df 출력 정답이 0 이라서 출력되는 row 가 없다.

# iterrow() 보다 33 배
# index 보다 13 배 빠르다.

0
0
[]


,enrollment_code,gender,bonus,penalty,school_code,result


- df.index/columns loop

```for idx in df.index```                       # index (각 row ) 별 계산   
```for idx in df.columns```                      # column 별 계산  
```for idx in df.unique_column_values_list```    # colum 값이 동일한 slicing 된 row 별 (그룹) 계산  

In [41]:

df2_3 = df.copy()

ans4 = 0

for idx in df2_3.index:
    if df2_3.at[idx, 'school_code'] in df2_3.at[idx, 'enrollment_code']:
        ans4 = ans4 +1
print(ans4)

# iterrow(), apply() 보다는 빠르지만, itertuples() 보다는 느리다. 
# loc 보다 at 이 2배 더 빠르다.

0


- 👉 apply()  
```
df.apply()        : axix = 0 :  각 열 (columns) 이 전달되어 행(row) 가 계산된다. (default)
                    axis = 1 :  각 행 (row) 이 전달되어 열(columns) 가 계산된다. 
                    
df.series.apply() : 행/열의 개별값이 전달된다. 
```

In [42]:
df2_4 = df.copy()

def funX (dfX :pd.DataFrame) -> bool:
    if dfX.school_code in dfX.enrollment_code:          # dfX.school_code 의 type 은 serise.school_code 이므로 str 개별값이다. 
        return True                                     # 전달된 row(행)의 df2_4['result'] 에 리턴할 값
    else:
        return False
        
df2_4['result'] = df2_4.apply(funX, axis=1)               # df에 적용되었으로, row (행) 순회하는 것이니까 axis=1 으로 row (행) 전달, column 계산

print(df2_4.head())
# print(type(df2_4.school_code))                          # df2_4.school_code 는 Series 인데????
# print(df2_4.info())
print(df2_4['result'].sum())                              # result 의 true 갯수
list(df2_4[df2_4['result']== True ].index)

  enrollment_code gender  bonus  penalty school_code  result
0          QEHRNZ      F    0.0      0.0        SWHS   False
1          IUGTFR      F    1.6      0.0        PYHS   False
2          PWJCQA      M    0.0      0.0        NKHS   False
3          ZYFHCX      F    5.5      0.0        NKHS   False
4          KIBAHX      M    0.6      0.0        MWHS   False
0


[]

### Q3. 학생들의 학업성취도를 분석하기 위해 군집화를 하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 등록 코드(enrollment_code)에 'HS'가 포함되어 있지 않은 학생들을 제거한다. 

👉 행 선별 (slicing)

drop 보다 slicing 으로 행을 가져오는 것이 더 간단하다.

In [43]:
df3 = df.copy()

print(df3.loc[df3.enrollment_code.str.contains('HS'),'enrollment_code'].count())

# df3.drop(df3.loc[~(df3.enrollment_code.str.contains('HS')),'enrollment_code'].index,inplace=True)
df3 = df3.loc[(df3.enrollment_code.str.contains('HS')),:]

661


In [44]:
df3.shape

(661, 5)

단계 2 : 성별(gender) 컬럼에서 "M"을 0, "F"를 1로 변환한다.  

 👉 df.serise.replace({ A:B, C:D })  
을   
df3.gender = np.where((df3.gender == 'M'), 0, 1)  
으로 바꿀수 있다.   
2개 이상인 경우 replace 사용  

In [45]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 661 entries, 212 to 86502
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   enrollment_code  661 non-null    object 
 1   gender           661 non-null    object 
 2   bonus            661 non-null    float64
 3   penalty          661 non-null    float64
 4   school_code      661 non-null    object 
dtypes: float64(2), object(3)
memory usage: 31.0+ KB


👉 df.series = np.where(df.series 조건, true, false)

repleace, slicing 이 유리할 때도 있다.

In [46]:
print(df3.loc[(df3.gender == 'M') | (df3.gender == "F"),'gender'].count())

# df3.gender.replace({'M':0, 'F':1},inplace=True) 
#  
# df3.loc[(df3.gender == 'M'),'gender'] = 0
# df3.loc[(df3.gender == "F"),'gender'] = 1
#
df3.gender = np.where((df3.gender == 'M'), 0, 1)   # 2개일 경우 유리 
print(df3.loc[(df3.gender == 'M') | (df3.gender == "F"),'gender'].count())

661
0


단계 3 : 독립 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2~9개 중 K-means Silhouette 를 통해 구하고, 이 중 두 번째로 높은 score를 최적의 K로 설정한다.  
- 독립 변수(총 3개) : 성별(gender), 상점(bonus), 벌점(penalty)  


In [47]:
df3.head()

,enrollment_code,gender,bonus,penalty,school_code
212,HSJZSV,1,2.7,0.0,NKHS
352,GHSBRB,0,1.7,0.0,PYHS
466,QCMHSO,0,1.9,0.0,HJHS
515,QCUGHS,1,0.6,0.0,PYHS
825,VFEHSR,1,0.0,1.0,HJHS


In [48]:
X_train = df3[['gender','bonus','penalty']]

silhouette_score(df,labels)

In [49]:
from sklearn.cluster import KMeans  
from sklearn.metrics import silhouette_score 

K = range(2,10)     # 2,3,4,5,6,7,8,9
si_list =[]

for k in K:
    my_si = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=229)
    my_si.fit(X_train)
    si_list.append(silhouette_score(X=X_train, labels=my_si.labels_))   # my_si.predict(X_train)

2번째로 큰 sil_coef 의 k 값을 찾는다. 

In [50]:
si_df = pd.DataFrame({ "K": K,
                      "sil_coef": si_list})
si_df.sort_values(by='sil_coef',ascending=False).iat[1,0]   # 과 iloc[1,0] 은 2번째 row 이지만 loc[1,0] 은  1번 row (3번째) 이다. 

8

In [51]:
si_df 

,K,sil_coef
0,2,0.492106
1,3,0.406293
2,4,0.367341
3,5,0.378843
4,6,0.381971
5,7,0.395668
6,8,0.408609
7,9,0.405521



단계 4 : 단계 3에서 최적의 K로 도출한 각 군집에 소속된 학생 수의 비율 중, 가장 큰 값을 구하시오  

※ 결과는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안예시 : 2.29)



KMean

In [52]:
my_KM = KMeans(n_clusters=8, init='k-means++', n_init=10, random_state=229)
my_KM.fit(X_train)
df3['cluster'] = my_KM.labels_

In [53]:
df3.head()

,enrollment_code,gender,bonus,penalty,school_code,cluster
212,HSJZSV,1,2.7,0.0,NKHS,4
352,GHSBRB,0,1.7,0.0,PYHS,0
466,QCMHSO,0,1.9,0.0,HJHS,0
515,QCUGHS,1,0.6,0.0,PYHS,2
825,VFEHSR,1,0.0,1.0,HJHS,5


In [54]:
df3.cluster.value_counts(normalize=True)#.iat[0].round(2)

# reset_index 해서 df[0] 하던가
# 바로 iat[0]


7    0.157337
4    0.154312
0    0.148260
6    0.142209
2    0.127080
3    0.122542
1    0.093797
5    0.054463
Name: cluster, dtype: float64

sort_values(asceding = False).iat[0]  

normalize (비율) 는 값을 정렬하여 표시 하므로 첫번째 값을 선택  (0번 at[0], loc[0] 이 아니다.)  
가장큰 값 찾기 max() 을 사용하자  
3번째로 큰값 찾기 : sort_values(asceding = False).iat[2]

👉 Reset_index(drop=True) 을 이용해서 row 0번을 찾을 수 있다.  
```df3['cluster'].value_counts(normalize=True).reset_index(drop=True)[0].round(2)```

In [56]:
df3['cluster'].value_counts(normalize=True).reset_index(drop=True)#[0].round(2)

0    0.157337
1    0.154312
2    0.148260
3    0.142209
4    0.127080
5    0.122542
6    0.093797
7    0.054463
Name: cluster, dtype: float64

#### 필요 라이브러리 함수,클래스 및 설정값 목록  

from sklearn.cluster import KMeans  
from sklearn.metrics import silhouette_score  
random_state = 229  
문제 지시 외 Default 값 사용  